### PLAN
1. Load up to Jun 2022 into BigQuery
2. Create github action to run BQ insert script nightly
3. Set grouped database table (history_df) in BigQuery to refresh nightly after insert
4. Make script to pull data history_df from BigQuery data

8. For streamlit app append [history_df, current_df, future_df] to make the combo_df
    - put history_df into cache using st.experimental_memo so its only called once each time app is opened

In [2]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit

In [ ]:
pull_nrg_data.release_token(accessToken)

In [16]:
# Release token
import certifi
import ssl
import http.client
server = 'api.nrgstream.com'

def release_token(accessToken):
    path = '/api/ReleaseToken'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()
    print('token released')

accessToken = 'GI3R5KS9k5PMXiD0s-iE69JDcH1T98THkIiTR3LXEjwvzGF86jaN_lZ8tL08_FsdM4ZpGYjFtRhnyvxvJoQMhkNiwRI6ybVAy22QTDhvKNGN2V0STLG2mhWiiAcZJnIVUAbbpgchsM-medLxM36QM8-5SwTOR1TwHNqKG24uWm5RDN8e2IhA1HrZ1fNGEEqCIl7-kMrw9idj5xyA9mqqXhGuK6IV9h27LTA7du08WPXOMkHqiM8Tna4Bs8bl8SALfeXdx-eA3EdRSpExjq4gShpyfXZREsQzSW5n86vMifcmqipOsesISLGRd_rYDbcmsKgoljBkATT8rORLbSplchbc9C0'
release_token(accessToken)

token released


In [ ]:
# Joining daily to hourly data
import pandas as pd

intraday = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='H'),data=[i for i in range(145)], columns=['hourly'])
daily = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='D'), data=[i for i in range(7)], columns=['daily'])
df = intraday.join(daily).fillna(method='ffill')
df

In [28]:
def getToken():
    username = st.secrets["nrg_username"]
    password = st.secrets["nrg_password"]
    server = 'api.nrgstream.com'
    tokenPath = '/api/security/token'
    tokenPayload = f'grant_type=password&username={username}&password={password}'
    headers = {"Content-type": "application/x-www-form-urlencoded"}
    # Connect to API server to get a token
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('POST', tokenPath, tokenPayload, headers)
    res = conn.getresponse()
    res_code = res.status
    # Check if the response is good
    if res_code == 200:
        res_data = res.read()
        # Decode the token into an object
        jsonData = json.loads(res_data.decode('utf-8'))
        accessToken = jsonData['access_token']
        # Calculate new expiry date
        tokenExpiry = datetime.now() + timedelta(seconds=5)
        #tokenExpiry = datetime.now() + timedelta(seconds=jsonData['expires_in'])
    elif res_code == 400:
        res.read()
        release_token(accessToken)
        getToken()
    else:
        res_data = res.read()
    conn.close()
    return accessToken, tokenExpiry

def release_token(accessToken):
    path = '/api/ReleaseToken'
    server = 'api.nrgstream.com'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()

def get_streamInfo(streamId):
    streamInfo = pd.read_csv('stream_codes.csv')
    streamInfo = streamInfo[streamInfo['streamId']==streamId]
    return streamInfo

def http_connect():
    server = 'api.nrgstream.com'
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server, context=context)
    return conn

def pull_data(fromDate, toDate, streamId, accessToken, tokenExpiry):
    # Setup the path for data request
    conn = http_connect()
    path = f'/api/StreamData/{streamId}?fromDate={fromDate}&toDate={toDate}'
    headers = {'Accept': 'Application/json', 'Authorization': f'Bearer {accessToken}'}
    conn.request('GET', path, None, headers)
    res = conn.getresponse()
    if res.code != 200:
        res.read()
        conn.close()
        release_token(accessToken)
        accessToken, tokenExpiry = getToken()
        pull_data(fromDate, toDate, streamId, accessToken, tokenExpiry)
    # Load json data & create pandas df
    else:
        jsonData = json.loads(res.read().decode('utf-8'))
        df = pd.json_normalize(jsonData, record_path='data')
        # Rename df cols
        df.rename(columns={0:'timeStamp', 1:'value'}, inplace=True)
        # Add streamInfo cols to df
        streamInfo = get_streamInfo(streamId)
        assetCode = streamInfo.iloc[0,1]
        streamName = streamInfo.iloc[0,2]
        fuelType = streamInfo.iloc[0,3]
        subfuelType = streamInfo.iloc[0,4]
        timeInterval = streamInfo.iloc[0,5]
        intervalType = streamInfo.iloc[0,6]
        df = df.assign(streamId=streamId, assetCode=assetCode, streamName=streamName, fuelType=fuelType, \
                        subfuelType=subfuelType, timeInterval=timeInterval, intervalType=intervalType)
        # Changing 'value' col to numeric and filling in NA's with previous value in col
        df.replace(to_replace={'value':''}, value=0, inplace=True)
        df['value'] = pd.to_numeric(df['value'])
        df.fillna(method='ffill', inplace=True)
        conn.close()
    return df

def current_data():
    tic = timeit.default_timer()
    streamIds = [86, 322684, 322677, 87, 85, 23695, 322665, 23694, 120, 124947, 122]
    realtime_df = pd.DataFrame([])
    today = datetime.now()
    for id in streamIds:
        accessToken, tokenExpiry = getToken()
        APIdata = pull_data(today.strftime('%m/%d/%Y'), today.strftime('%m/%d/%Y'), id, accessToken, tokenExpiry)
        APIdata['timeStamp'] = pd.to_datetime(APIdata['timeStamp'])
        realtime_df = pd.concat([realtime_df, APIdata], axis=0)
        release_token(accessToken)
    toc = timeit.default_timer()
    print(toc - tic)
    return realtime_df, today

realtime_df, today = current_data()
realtime_df

5.227837817998079


,timeStamp,value,streamId,assetCode,streamName,fuelType,subfuelType,timeInterval,intervalType
0,2022-08-18 00:00:00,1179,86,NaN,Coal,Coal,Coal,5 min,supply
1,2022-08-18 00:05:00,1195,86,NaN,Coal,Coal,Coal,5 min,supply
2,2022-08-18 00:10:00,1222,86,NaN,Coal,Coal,Coal,5 min,supply
3,2022-08-18 00:15:00,1236,86,NaN,Coal,Coal,Coal,5 min,supply
4,2022-08-18 00:20:00,1245,86,NaN,Coal,Coal,Coal,5 min,supply
...,...,...,...,...,...,...,...,...,...
199,2022-08-18 16:35:00,49,122,NaN,Saskatchewan,Saskatchewan,Saskatchewan,5 min,supply
200,2022-08-18 16:40:00,49,122,NaN,Saskatchewan,Saskatchewan,Saskatchewan,5 min,supply
201,2022-08-18 16:45:00,49,122,NaN,Saskatchewan,Saskatchewan,Saskatchewan,5 min,supply
202,2022-08-18 16:50:00,49,122,NaN,Saskatchewan,Saskatchewan,Saskatchewan,5 min,supply
